In [1]:
import os

In [2]:
%pwd

'd:\\Kidney_tumor_detection\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Kidney_tumor_detection'

MLFLOW_TRACKING_URI=https://dagshub.com/shrianshsingh20/Kidney_tumor_Detection.mlflow \
MLFLOW_TRACKING_USERNAME=shrianshsingh20 \
MLFLOW_TRACKING_PASSWORD=1ed862e5af9a1dde281263c79e0305550ac28646 \
python script.py

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/shrianshsingh20/Kidney_tumor_Detection.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="shrianshsingh20"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1ed862e5af9a1dde281263c79e0305550ac28646"

BASICALLY WE ARE ADDING THESE CREDINTIAL IN OUR ENVIRONMENT
In Python, the os.environ module is part of the os module, and it is used for interacting with the environment variables of the operating system

In [6]:
import tensorflow as tf

In [7]:
model=tf.keras.models.load_model('artifacts/training/model.h5')

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from src.utils import *
from src.constants import *


In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            all_params=self.params,
            mlflow_uri="https://dagshub.com/shrianshsingh20/Kidney_tumor_Detection.mlflow",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


if tracking_url_type_store != "file":
In this case, it means that the MLflow tracking is not based on a local file system but is likely a remote tracking URI (e.g., a database or cloud storage).
The code logs the Keras model using mlflow.keras.log_model.
It specifies the model name as "model" and additionally provides the registered_model_name parameter as "VGG16Model". This registers the model with the Model Registry using the specified name.

If the tracking URI type is "file":

This branch is executed when the tracking URI is a local file system.
The code logs the Keras model using mlflow.keras.log_model, specifying the model name as "model".
In this case, the model is logged without registering it with the Model Registry. The Model Registry typically works with remote storage, and local file storage might not support Model Registry features.

In summary, this conditional statement determines whether the machine learning model should be registered with the Model Registry based on the type of MLflow tracking URI. If the tracking URI is remote, it registers the model with a specified name; otherwise, it simply logs the model without registration.

In [13]:
try:
     
    config=ConfigurationManager(CONFIG_FILE_PATH,PARAM_FILE_PATH)
    eval_config=config.get_evaluation_config()
    evaluation1=Evaluation(eval_config)
    evaluation1.evaluation()
    evaluation1.log_into_mlflow()
except Exception as e:
    raise CustomException(e,sys)


Found 139 images belonging to 2 classes.
9/9 [==============================] - 39s 4s/step - loss: 18.0490 - accuracy: 0.4820


2023/11/29 19:25:27 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\SHRIAN~1\AppData\Local\Temp\tmpkjer8hy_\model\data\model\assets


d:\Kidney_tumor_detection\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/11/29 19:26:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
